# BUSINESS CONTEXT

**Company Name:** DHAUZ Real State

**Product/Service:** The company offers a service that estimates apartments price through their features and characteristics, serving as a guide to customers wanting to sell their apartments.

**Business Model:**

- The service price is based on two components:

    - 1 - A minimum price per estimate, which is R$ 200,00
    - 2 - Based on the error of the estimate, which is based on the MAPE. If the error is less than 0.05, it is added R$ 400,00. For every 1% more error, it is subtracted R$ 20,00. If the error is greater than 20%, than the customer only pays the minimun price

**Current Situation:**

- The estimate is done by real state agent
- The current MAPE is around 18%, varying from 30% and 8%

**Problem:**

- The company started this business model to validade their service, and now is trying to improve their profit, delivering a more accurate estimate.

**Proposed Solution:**

- Create a Linear Model to estimate apartment price, to achieve a more accurate estimate and helping real state agents do their job.

**Main Questions to be answered:**

- How much is the average error?
- How much more profit DHAUZ Real State will make with the proposed solution?

# IMPORTS

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime

# NOTEBOOK CONFIGS

In [3]:
pd.set_option('display.max_columns', None)

# HELPER FUNCTIONS

# 1.0 - LOADING DATA

In [31]:
df_raw = pd.read_csv(r'C:\Users\AlvaroMatsuda\OneDrive - DHAUZ\Área de Trabalho\DHAUZ_Files\Vivareal_Webscraping\scraped_data\df_webscraped.csv').drop(columns='Unnamed: 0')
df_raw.head()

,endereco,area,quartos,banheiros,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte,scrap_datetime
0,"Rua Francisco Pedro do Amaral, 251 - Jaguaré, ...",68m²,3 quartos,2 banheiros 1 suíte,2 vagas,R$ 500.000,R$ 575,"Piscina, Churrasqueira, Elevador, Academia, Ac...",2548091356,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
1,"Rua Santo Antônio, 670 - Bela Vista, São Paulo...",44m²,1 quarto,1 banheiro,Não informado Solicitar,R$ 289.000,R$ 319,"Aceita animais, Elevador, Portão eletrônico",2552171953,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
2,"Rua Domingos Rodrigues, 509 - Lapa, São Paulo ...",54m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 550.000,R$ 800,"Piscina, Mobiliado, Churrasqueira, Elevador, A...",2544567013,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
3,"Rua Humberto I, 193 - Vila Mariana, São Paulo ...",172m²,4 quartos,5 banheiros 3 suítes,4 vagas,R$ 4.810.000,R$ 2.100,"Piscina, Churrasqueira, Academia, Aceita anima...",2543253547,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
4,"Rua Quariteré, 267 - Parque da Mooca, São Paul...",78m²,2 quartos,2 banheiros 1 suíte,1 vaga,R$ 680.000,R$ 1.000,"Piscina, Churrasqueira, Elevador, Condomínio f...",2550615565,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947


# 2.0 - DATA CLEANING

In [203]:
df02 = df_raw.copy()

In [204]:
# =============== 1.0 - area ===============
# Remove "m²"
df02['area'] = df02['area'].str.strip('m²')

# Drop "sem metrage"
df02.drop(df02[df02['area'] == 'sem metrage'].index, inplace=True)

# Convert to int
df02['area'] = df02['area'].astype('int64')

# =============== 2.0 - quartos ===============
# remove "quartos"
df02['quartos'] = df02['quartos'].str.strip('quartos')

# Convert to int
df02['quartos'] = df02['quartos'].astype('int64')

# =============== 3.0 - suite ===============
# create suite column
df02.insert(4, 'suite', df02[df02['banheiros'].str.contains('suíte')]['banheiros'].str.replace(' ', '').str[9:12].str.strip('suo'))

# fill NA - suite
df02['suite'].fillna(0, inplace=True)

# Convert to int
df02['suite'] = df02['suite'].astype('int64')

# =============== 4.0 - banheiros ===============

# =============== 5.0 - vaga_garagem ===============

# =============== 6.0 - valor_aluguel ===============

# =============== 7.0 - valor_cond ===============

# =============== 8.0 - caracteristicas ===============



In [207]:
df02

,endereco,area,quartos,banheiros,suite,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte,scrap_datetime
0,"Rua Francisco Pedro do Amaral, 251 - Jaguaré, ...",68,3,2 banheiros 1 suíte,1,2 vagas,R$ 500.000,R$ 575,"Piscina, Churrasqueira, Elevador, Academia, Ac...",2548091356,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
1,"Rua Santo Antônio, 670 - Bela Vista, São Paulo...",44,1,1 banheiro,0,Não informado Solicitar,R$ 289.000,R$ 319,"Aceita animais, Elevador, Portão eletrônico",2552171953,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
2,"Rua Domingos Rodrigues, 509 - Lapa, São Paulo ...",54,2,2 banheiros 1 suíte,1,1 vaga,R$ 550.000,R$ 800,"Piscina, Mobiliado, Churrasqueira, Elevador, A...",2544567013,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
3,"Rua Humberto I, 193 - Vila Mariana, São Paulo ...",172,4,5 banheiros 3 suítes,3,4 vagas,R$ 4.810.000,R$ 2.100,"Piscina, Churrasqueira, Academia, Aceita anima...",2543253547,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
4,"Rua Quariteré, 267 - Parque da Mooca, São Paul...",78,2,2 banheiros 1 suíte,1,1 vaga,R$ 680.000,R$ 1.000,"Piscina, Churrasqueira, Elevador, Condomínio f...",2550615565,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
...,...,...,...,...,...,...,...,...,...,...,...,...
10555,"Rua Eugênio de Medeiros - Pinheiros, São Paulo...",60,2,2 banheiros 1 suíte,1,2 vagas,R$ 1.050.000,R$ 549,sem informacao,2542503492,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
10556,"Lauzane Paulista, São Paulo - SP",93,2,2 banheiros 1 suíte,1,2 vagas,R$ 700.000,R$ 679,"Garagem, Varanda, Cozinha",2551040585,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
10557,"Rua Ibirajá - Vila Guarani, São Paulo - SP",53,2,1 banheiro 1 suíte,1,1 vaga,R$ 519.000,R$ 650,"Piscina, Churrasqueira, Elevador, Condomínio f...",2547621336,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
10558,"Rua São Jorge, 125 - Tatuapé, São Paulo - SP",66,2,2 banheiros 1 suíte,1,2 vagas,R$ 539.000,R$ 700,"Piscina, Mobiliado, Churrasqueira, Elevador, C...",2539189788,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947


In [193]:
df02[df02['banheiros'].str.contains('suíte')]['banheiros'].str.replace(' ', '').str[10:11]

0        1
2        1
3        3
4        1
5        2
        ..
10555    1
10556    1
10557    s
10558    1
10559    3
Name: banheiros, Length: 6304, dtype: object

In [181]:
df02[df02['banheiros'].str.replace(' ', '').str.contains('suíte')]['banheiros'].str[15:17]

0         1
2         1
3         3
4         1
5         2
         ..
10555     1
10556     1
10557    1 
10558     1
10559     3
Name: banheiros, Length: 6304, dtype: object

In [179]:
text = '     2 banheiros     1 suíte'
print(text.replace(' ', ''))

2banheiros1suíte


In [172]:
df02['banheiros']

0         2 banheiros     1 suíte
1                      1 banheiro
2         2 banheiros     1 suíte
3        5 banheiros     3 suítes
4         2 banheiros     1 suíte
                   ...           
10555     2 banheiros     1 suíte
10556     2 banheiros     1 suíte
10557      1 banheiro     1 suíte
10558     2 banheiros     1 suíte
10559    3 banheiros     3 suítes
Name: banheiros, Length: 10559, dtype: object

In [141]:
df02[df02['banheiros'].str.strip.contains('suíte')]['banheiros'].str[15:17]

0         1
2         1
3         3
4         1
5         2
         ..
10555     1
10556     1
10557    1 
10558     1
10559     3
Name: banheiros, Length: 6304, dtype: object

In [153]:
df02.iloc[[2225, 8265], :]['banheiros'][2226]

'10 banheiros     5 suítes'

In [138]:
df02[df02['suite'] == '']

,endereco,area,quartos,banheiros,suite,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte,scrap_datetime
2226,"Rua Jaime Costa, 425 - Retiro Morumbi , São Pa...",1250,5,10 banheiros 5 suítes,,12 vagas,R$ 24.990.000,R$ 19.510,"Piscina, Mobiliado, Churrasqueira, Elevador, C...",2538655767,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
8266,"Rua Seridó - Jardim Europa, São Paulo - SP",1980,7,11 banheiros 7 suítes,,Não informado Solicitar,R$ 70.000.000,R$ 25.000,"Piscina, Churrasqueira, Varanda, Academia, Ar-...",2550930730,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947


In [132]:
df02['suite'].str.strip()

0          1
1        NaN
2          1
3          3
4          1
        ... 
10555      1
10556      1
10557      1
10558      1
10559      3
Name: suite, Length: 10559, dtype: object

In [111]:
x = df02[df02['suite'] == " "]
x['banheiros'].str.contains('suíte')['banheiros'].str

,endereco,area,quartos,banheiros,suite,vaga_garagem,valor_aluguel,valor_cond,caracteristicas,id_anuncio,fonte,scrap_datetime
28,"Rua Dona Antônia de Queirós - Consolação, São ...",38,1,1 banheiro 1 suíte,,1 vaga,R$ 549.000,R$ 600,"Piscina, Churrasqueira, Elevador, Condomínio f...",2551430565,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
67,"Rua Dona Antônia de Queirós - Consolação, São ...",38,1,1 banheiro 1 suíte,,1 vaga,R$ 549.000,R$ 600,"Piscina, Churrasqueira, Elevador, Condomínio f...",2551430565,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
70,"Rua Ibirajá - Vila Guarani, São Paulo - SP",53,2,1 banheiro 1 suíte,,1 vaga,R$ 519.000,R$ 650,"Piscina, Churrasqueira, Elevador, Condomínio f...",2547621336,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
106,"Rua Ibirajá - Vila Guarani, São Paulo - SP",53,2,1 banheiro 1 suíte,,1 vaga,R$ 519.000,R$ 650,"Piscina, Churrasqueira, Elevador, Condomínio f...",2547621336,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
145,"Rua Gomes de Carvalho, 1146 - Vila Olímpia, Sã...",62,1,1 banheiro 1 suíte,,1 vaga,R$ 1.200.000,R$ 950,"Piscina, Condomínio fechado, Academia, Aceita ...",2499606686,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
...,...,...,...,...,...,...,...,...,...,...,...,...
10483,"Rua Nelson Gama de Oliveira, 850 - Morumbi, Sã...",66,2,1 banheiro 1 suíte,,1 vaga,R$ 499.990,sem informacao,"Piscina, Elevador, Varanda, Academia, Playgrou...",2549566677,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
10500,"Rua do Estilo Barroco, 633 - Brooklin, São Pau...",41,2,1 banheiro 1 suíte,,Não informado Solicitar,R$ 395.000,R$ 430,"Piscina, Churrasqueira, Elevador, Condomínio f...",2538535398,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
10505,"Alameda Jaú, 1607 - Jardins, São Paulo - SP",43,1,1 banheiro 1 suíte,,1 vaga,R$ 300.000,R$ 1,"Piscina, Mobiliado, Elevador, Academia, Ar-con...",2506964935,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947
10517,"Rua Ibirajá - Vila Guarani, São Paulo - SP",53,2,1 banheiro 1 suíte,,1 vaga,R$ 519.000,R$ 650,"Piscina, Churrasqueira, Elevador, Condomínio f...",2547621336,https://www.vivareal.com.br/imovel/apartamento...,2022-03-02 23:08:39.136947


In [68]:
df02.dtypesa

endereco           object
area                int64
quartos             int64
banheiros          object
vaga_garagem       object
valor_aluguel      object
valor_cond         object
caracteristicas    object
id_anuncio          int64
fonte              object
scrap_datetime     object
dtype: object

# 3.0 - DATA DESCRIPTION

## 3.1 - Rename Columns

## 3.2 - Data Dimentions

## 3.3 - Data Types

## 3.4 - Check NAs

## 3.5 - Descriptive Statistics

### 3.5.1 - Numerical Variables

### 3.5.2 - Categorical Variables

# 4.0 - CHECK OUTLIERS

# 5.0 - Hypothesis

## 5.1 - Mind Map Hypothesis

## 5.2 - Hypothesis Creation

# 6.0 - FEATURE ENGINEERING

# 7.0 - EDA

## 7.1 - Univariate Analysis

### 7.1.1 - Target Variable

### 7.1.2 - Numerical Variables

### 7.1.3 - Categorial Variables

## 7.2 - Bivariate Analysis

In [ ]:
# Validate Hypothesis

## 7.3 - Multivariate Analysis

### 7.3.1 - Numerical Variables

### 7.3.2 - Categorical Variables

# 8.0 - DATA PREPARATION

## 8.1 - Data Filtering

### 8.1.1 - Remove Outliers

### 8.1.2 - Drop Unnecessary Rows and Columns

## 8.2 - Train and Test Split

## 8.3 - Rescale/Encode Variables

### 8.3.1 - Train Dataset

### 8.3.2 - Test Dataset

# 9.0 - FEATURE SELECTION

# 10.0 - MACHINE LEARNING MODELLING

## 10.1 - Baseline Model

In [ ]:
# Usar a média do valor/m² por região/bairro como Baseline

## 10.2 - Modelos (Colocar o nome dos modelos)

## 10.3 - Models Performance - Comparison

# 11.0 - CROSS-VALIDATION SCORING

# 12.0 - HYPERPARAMETER FINE TUNNING

## 12.1 - Final Model with Best Parameters

# 13.0 - TRANSLATION AND INTERPRETATION OF THE ERROR

## 13.1 - Business Performance

## 13.2 - Machine Learning Performance on Test Dataset

# 14.0 - DEPLOY MODEL TO PRODUCTION

## 14.1 - Vivareal Class

## 14.2 - API Handler

## 14.3 - API Tester